In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime as dt
import requests
import numpy as np
import pandas as pd

In [3]:
artarr = np.zeros((210, 4), dtype='>U60')
artinfo = np.zeros((210, 1), dtype='>U2000')

In [4]:
def selection(driver, addrow, i, syllable):
    """Move into detailed info of each artists then crawl them"""
    driver.implicitly_wait(10)

    artistClick = driver.find_element_by_css_selector('#pageList > div.list_atist12.d_artist_list > ul > li:nth-child('+ str(i) +') > div > div > dl > dt > a')
    artistClick.click()

    driver.implicitly_wait(10)

    detailinfo = driver.find_element_by_css_selector('#conts > div.wrap_tab_atist > ul > li:nth-child(2) > a')
    detailinfo.click()

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    artist = soup.select('#conts > div.wrap_dtl_atist > div > div.wrap_atist_info > p')
    features_dt = soup.select('#conts > div.section_atistinfo03 > dl > dt')
    features = soup.select('#conts > div.section_atistinfo03 > dl > dd')
    introduce = soup.select('#d_artist_intro')
    
    artistproc1 = artist[0].text.strip().replace("아티스트명","")
    artistproc2 = artistproc1.replace(u"\xa0", u" ")
    artarr[5*syllable+i+addrow, 0] = artistproc2
    
    try: 
        introproc1 = introduce[0].text.strip()
        introproc2 = introproc1.replace(u"\xa0", u" ")
        introproc3 = introproc2.replace(u"\t", u" ")
        artinfo[5*syllable+i+addrow, 0] = introproc3
    except IndexError:
        pass
  
    j = 1
    if features_dt[0].text.strip() == "데뷔":
        if features_dt[2].text.strip() == "결성일":
            if features_dt[3].text.strip() != "해체일":
                for k in [1, 3, 4]:
                    feature = features[k].text.strip().split('\n')
                    artarr[5*syllable+i+addrow, j] = ''.join(feature).replace("\t", "")
                    j = j + 1
            else:
                for k in [1, 4, 5]:
                    feature = features[k].text.strip().split('\n')
                    artarr[5*syllable+i+addrow, j] = ''.join(feature).replace("\t", "")
                    j = j + 1
        else:
            for k in range(1, 4):
                feature = features[k].text.strip().split('\n')
                artarr[5*syllable+i+addrow, j] = ''.join(feature).replace("\t", "")
                j = j + 1
    elif features_dt[0].text.strip() == "활동년대":
         for k in range(0, 3):
            feature = features[k].text.strip().split('\n')
            artarr[5*syllable+i+addrow, j] = ''.join(feature).replace("\t", "")
            j = j + 1

    driver.back()
    driver.back()

In [5]:
def korean(driver):
    """Crawl from ㄱ to ㅎ, Numbers and others"""
    for syllable in range(1, 17):
        for i in range(1, 6):
            click = driver.find_element_by_css_selector('#conts > div.wrap_finder.atist > dl > dd > div.wrap_btn_atist.first_child > button:nth-child('+str(syllable)+')')
            click.click()
            #((syllable-1)*5) + i-1
            selection(driver, -6, i, syllable)

In [6]:
def english(driver):
    """Crawl from A to Z"""
    for syllable in range(1, 27):
        for i in range(1, 6):
            click = driver.find_element_by_css_selector('#conts > div.wrap_finder.atist > dl > dd > div:nth-child(2) > button:nth-child('+str(syllable)+') > span > span')
            click.click()
            #((syllable-1)*5) + i-1 + 80
            selection(driver, 74, i, syllable)

In [7]:
def crawlArtistFeatures(url):
    """Crawl top 5 Artists' name, year, type and genre for each syllable"""
    res = requests.get(url, headers={"User-agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0 rv:11.0) like Gecko"})
    res.raise_for_status()
    driver = webdriver.Chrome()
    driver.get(url)
    
    korean(driver)
    english(driver)

    driver.quit()

In [8]:
crawlArtistFeatures('https://www.melon.com/artistplus/finder/index.htm')

In [9]:
artarr[1:5, :]

array([['김필', '2010 년대', '솔로|남성',
        '발라드, 댄스, R&B/Soul, 인디음악, 록/메탈, 포크/블루스'],
       ['김소유 (김소희)', '2010 년대', '솔로|여성', '성인가요'],
       ['김하온 (HAON)', '2010 년대', '솔로|남성', '발라드, 랩/힙합'],
       ['김동률', '1990, 2000, 2010 년대', '솔로|남성', '발라드']], dtype='>U60')

In [10]:
artinfo[1:5]

array([['호소력 짙은 "고드름 보이스" 김필은 2014년 Mnet에서 방영된 오디션 프로그램 "슈퍼스타 K 시즌 6"에 참가해 준우승을 거두면서 일약 스타덤에 올랐다.  2015년 6월 첫 미니앨범 [Feel Free]을 발매, 싱어송라이터로서 솔직하고 자유로운 이야기를 담아냈고. 두번째 미니앨범 [from Feel]에서는 진중한 마음속의 이야기를 꺼냈다.  이후 2년간의 공백기 끝에, 첫 싱글인 "목소리"에서는 기다려준 팬들에 대한 감사와 사랑을 노래하며, 앞으로 본격적인 활동이 시작됨을 알렸다.    김필은 많은 오리지널 사운드 트랙에 이름을 올렸는데, "펀치", "응답하라 1988"의 \'청춘\', "내일 그대와", "어비스" 등 많은 OST가 대중의 사랑을 받았다.  또한 김필의 복면가왕 미방송분인 유튜브 영상은 1000만뷰를 돌파하며 젊은 대중들의 사랑을 받으며, 앞으로의 활동에 기대감을 실어 주었다.'],
       ['김소유는 대한민국의 트로트 가수이다. 2015년 봄 이별과 사랑에 인생사를 표현한 정규 1집 [사랑 방정식]으로 데뷔하였고, 싱글 [초생달]로 활동을 이어갔다. 2019년 TV조선의 트로트 오디션 프로그램 "내일은 미스트롯"에 참가해 \'십분내로\', \'진정인가요\'등의 곡으로 우수한 실력을 뽐내 대중에게 눈도장을 찍었으며, 그 해 가을 사랑과 사람 이야기를 표현해낸 정통 트로트 싱글 [별 아래 산다]를 선보였다.'],
       ['"명상을 즐기는 래퍼"라는 독특한 바이브를 자랑하는 김하온은 "고등래퍼 시즌2"의 최종 우승자이다. 지원자 시절에는 프로그램이 진행되는 동안 가장 많은 주목을 받은 기대주이기도 했다. 나이답지 않은 철학적인 가사가 특징이며, 발음을 흘릴 때는 흘리고 좋은 딕션이 필요할 때는 귀에 잘 들리는 랩을 하는 등 양면을 완벽하게 오가며 스킬적인 면에서도 호의적인 평가를 받는다.같은 출연자인 김민규, 김윤호, 방재민, 배연서, 이병재, 임수와 함께 10대 힙합크루인 키프클랜 소속이다. "고등래퍼 시즌 

In [598]:
# Data preprocessing
bdf = pd.DataFrame(artarr, columns=['artists', 'year', 'type', 'genre'])
bdf['info'] = artinfo
bdf.set_index('artists', inplace=True)
df1 = bdf[(bdf['year']!='')]
df = df1[(df1['info']!='')]
df.shape
# df.isnull().values.any()

(197, 4)

In [599]:
# Change Korean to English language in DataFrame
pd.options.mode.chained_assignment = None
df['year'] = df['year'].str.replace(' 년대', '')
df['type'] = df['type'].str.replace('|', ', ')
df['type'] = df['type'].str.replace('솔로', 'Solo')
df['type'] = df['type'].str.replace('그룹', 'Group')
df['type'] = df['type'].str.replace('남성', 'Male')
df['type'] = df['type'].str.replace('여성', 'Female')
df['type'] = df['type'].str.replace('혼성', 'Both')
df['genre'] = df['genre'].str.replace('발라드', 'Ballad')
df['genre'] = df['genre'].str.replace('댄스', 'Dance')
df['genre'] = df['genre'].str.replace('랩/힙합', 'Rap/Hiphop')
df['genre'] = df['genre'].str.replace('인디음악', 'Indie')
df['genre'] = df['genre'].str.replace('록/메탈', 'Rock/Metal')
df['genre'] = df['genre'].str.replace('포크/블루스', 'Folk/Blues')
df['genre'] = df['genre'].str.replace('국내CCM', 'KoreanCCM')
df['genre'] = df['genre'].str.replace('뉴에이지', 'Newage')
df['genre'] = df['genre'].str.replace('일렉트로니카', 'Electronica')
df['genre'] = df['genre'].str.replace('포크', 'Folk')
df['genre'] = df['genre'].str.replace('재즈', 'Jazz')
df['genre'] = df['genre'].str.replace('성인가요', 'Trot')
df['genre'] = df['genre'].str.replace('월드뮤직', 'World')
df['genre'] = df['genre'].str.replace('컨트리', 'Country')
df['genre'] = df['genre'].str.replace('블루스', 'Blues')
df['genre'] = df['genre'].str.replace('워십', 'Worship')
df['genre'] = df['genre'].str.replace('클래식', 'Classics')
df['genre'] = df['genre'].str.replace('크로스오버', 'Crossover')
df['genre'] = df['genre'].str.replace('보컬Jazz', 'VocalJazz')
df['genre'] = df['genre'].str.replace('애시드/퓨전/팝', 'Acid/Fusion/Pop')
df['genre'] = df['genre'].str.replace('보사노바', 'BossaNova')
df['genre'] = df['genre'].str.replace('국외', 'ForeignCCM')
df['genre'] = df['genre'].str.replace('만화', 'Comedy')
df['genre'] = df['genre'].str.replace('성악', 'Vocal')
df['genre'] = df['genre'].str.replace('오페라', 'Opera')
df['genre'] = df['genre'].str.replace('중국음악', 'Chinese')
df['genre'] = df['genre'].str.replace('키즈', 'Kids')
df['genre'] = df['genre'].str.replace('\(.*\)', '', regex = True)
df.sample(20)

,year,type,genre,info
artists,,,,
yaeji,2010,"Solo, Female",Electronica,yaeji는 GØDMØDE MUSIC 레이블에 소속되어 있는 DJ이자 일렉트로니카 ...
5 Seconds Of Summer,2010,"Group, Male","POP, Rock/Metal, Electronica, Country, EDM",5 Seconds Of Summer는 오스트레일리아의 팝 록 밴드이다. Luke H...
SHINee (샤이니),"2000, 2010","Group, Male","Ballad, Dance, R&B/Soul, Electronica, Classics...",샤이니는 SM엔터테인먼트에서 컨템퍼러리 밴드(Contemporary Band)라는 ...
Nicki Minaj (Onika Tanya Maraj),2010,"Solo, Female","POP, Electronica, Rap/Hiphop, R&B/Soul",Nicki Minaj는 미국의 래퍼이자 싱어송라이터이다. 2007년 [Playtim...
#안녕,2010,"Solo, Male","Ballad, Indie, Folk/Blues","더필름의 객원 싱어로 출발해, '하나', '우리 다시 연락해요'를 히트시킨 #안녕은..."
방탄소년단,2010,"Group, Male","Ballad, Dance, Rap/Hiphop, R&B/Soul, Rock/Metal",가요계의 새로운 역사를 쓰고 있는 그룹 방탄소년단은 데뷔 이후 끊임없는 성장세를 보...
Marshmello,2010,"Solo, Male","POP, Electronica, Rap/Hiphop, R&B/Soul, EDM",Marshmello는 미국의 프로듀서이자 DJ이다. 2015년 자신의 첫 오리지널 ...
NCT DREAM,2010,"Group, Male","Ballad, Dance, Rap/Hiphop, R&B/Soul, Rock/Metal","NCT DREAM은 SM 엔터테인먼트에 소속되어 있는 천러(CHENLE), 런쥔(R..."
V,2010,"Solo, Male",Dance,V는 빅히트 엔터테인먼트에 소속된 7인조 보이그룹 방탄소년단의 멤버로 그룹의 서브 ...


In [600]:
# df['info'] = df['info'].map(lambda x: x.split('.'))
df['info'].dropna(inplace=True)
listinfo = df['info'].tolist()
for i in range(len(listinfo)):
    listinfo[i] = listinfo[i].split('.')
    if len(listinfo[i]) >= 2:
        listinfo[i] = listinfo[i][:-1]
listinfo[49]

['키 (Key)는 SM 엔터테인먼트에 소속된 보이 그룹 SHINee (샤이니)의 멤버이다',
 ' 2005년 SM 엔터테인먼트에서 실시한 전국 투어 오디션에서 무려 8,000 대 1의 경쟁률을 뚫고 합격해 SM의 연습생이 되었고, 연습생 3년 만인 2008년 5월 그룹의 첫 미니 앨범 [누난 너무 예뻐(Replay)]를 발표하며 정식 데뷔했다',
 " 타이틀곡 '누난 너무 예뻐'는 선풍적인 인기로 단숨에 스타 반열에 올랐으며, 이후 '산소 같은 너', '줄리엣', 'Ring Ding Dong', 'Lucifer', 'Sherlock', 'Dream Girl', 'Everybody' 등 수많은 히트곡을 배출했다",
 "   키 (Key)는 2009년 샤이니 미니앨범 3집 [2009, Year Of Us]부터 작사에 참여하는데, 여기에 수록된 'Get Down'이 그의 곡이다",
 " 정규 2집 [LUCIFER]에서는 '악(Shout Out)'을, 리패키지 앨범 [HELLO]에서는 'Get It'이라는 곡을 수록했다",
 ' 2014년 3월에는 동갑내기 친구 인피니트의 우현과 함께 유닛 투하트를 결성해 미니 앨범 [1st Mini Album]을 발매했다',
 '   2012년부터는 뮤지컬 배우로도 활동하는데, "캐치 미 이프 유 캔"과 "보니 앤 클라이드", "삼총사" 등의 작품에 출연했다',
 ' 이 작품들은 SM의 선배들도 연기한 바 있는 캐릭터로 키 (Key)는 선배들의 연기에 뒤지지 않는다는 평가로 연기자의 가능성을 인정받았다',
 ' 이후 활발한 그룹 활동을 이어갔으며, 2018년 11월 고대하던 첫 번째 솔로 정규작 [FACE]를 발매해 다채로운 10곡을 담아냈다',
 " 4개월 뒤에는 (여자)아이들의 소연이 피처링으로 참여한 타이틀곡 'I Wanna Be'와 신곡 두 곡을 포함한 리패키지 앨범으로 다시 컴백해 감각적인 음악 세계를 선보이며 팬들의 기대에 부응했다"]

In [601]:
from krwordrank.hangle import normalize
for i in range(len(listinfo)):
    for j in range(len(listinfo[i])):
        listinfo[i][j] = normalize(str(listinfo[i][j]), english=True, number=True)
listinfo[49]

['키 Key 는 SM 엔터테인먼트에 소속된 보이 그룹 SHINee 샤이니 의 멤버이다',
 '2005년 SM 엔터테인먼트에서 실시한 전국 투어 오디션에서 무려 8 000 대 1의 경쟁률을 뚫고 합격해 SM의 연습생이 되었고 연습생 3년 만인 2008년 5월 그룹의 첫 미니 앨범 누난 너무 예뻐 Replay 를 발표하며 정식 데뷔했다',
 '타이틀곡 누난 너무 예뻐 는 선풍적인 인기로 단숨에 스타 반열에 올랐으며 이후 산소 같은 너 줄리엣 Ring Ding Dong Lucifer Sherlock Dream Girl Everybody 등 수많은 히트곡을 배출했다',
 '키 Key 는 2009년 샤이니 미니앨범 3집 2009 Year Of Us 부터 작사에 참여하는데 여기에 수록된 Get Down 이 그의 곡이다',
 '정규 2집 LUCIFER 에서는 악 Shout Out 을 리패키지 앨범 HELLO 에서는 Get It 이라는 곡을 수록했다',
 '2014년 3월에는 동갑내기 친구 인피니트의 우현과 함께 유닛 투하트를 결성해 미니 앨범 1st Mini Album 을 발매했다',
 '2012년부터는 뮤지컬 배우로도 활동하는데 캐치 미 이프 유 캔 과 보니 앤 클라이드 삼총사 등의 작품에 출연했다',
 '이 작품들은 SM의 선배들도 연기한 바 있는 캐릭터로 키 Key 는 선배들의 연기에 뒤지지 않는다는 평가로 연기자의 가능성을 인정받았다',
 '이후 활발한 그룹 활동을 이어갔으며 2018년 11월 고대하던 첫 번째 솔로 정규작 FACE 를 발매해 다채로운 10곡을 담아냈다',
 '4개월 뒤에는 여자 아이들의 소연이 피처링으로 참여한 타이틀곡 I Wanna Be 와 신곡 두 곡을 포함한 리패키지 앨범으로 다시 컴백해 감각적인 음악 세계를 선보이며 팬들의 기대에 부응했다']

In [602]:
# Keyword extraction
from krwordrank.word import KRWordRank

In [603]:
min_count = 1   
max_length = 10 
beta = 0.85    
max_iter= 10
verbose = False
keywordlists = []
for i in range(len(listinfo)):
    artistkeyword = []
    wordrank_extractor = KRWordRank(min_count, max_length)
    keywords, rank, graph = wordrank_extractor.extract(listinfo[i], beta, max_iter, verbose)
    for word in keywords.items():
        artistkeyword.append(word[0])
    keywordlists.append(artistkeyword)

In [604]:
len(keywordlists)

197

In [605]:
df['info_keywords'] = ''
for i in range(len(keywordlists)):
    try:
        for j in range(30):
            df['info_keywords'][i] = df['info_keywords'][i] + keywordlists[i][j] + ','
    except IndexError:
        pass

In [606]:
df.to_csv("artists.csv", encoding='utf-8')

In [628]:
# data_type = {'artists': 'str', 'year': 'str', 'type': 'str', 'genre': 'str'}
df = pd.read_csv("artists.csv", encoding = "utf-8", index_col='artists')
df.shape

(197, 5)

In [629]:
df['year'].astype('str')
df['year'] = df['year'].map(lambda x: x.split(','))
df['type'] = df['type'].map(lambda x: x.split(','))
df['genre'] = df['genre'].map(lambda x: x.split(','))
df['info_keywords'] = df['info_keywords'].map(lambda x: x.split(','))
for i in range(df.shape[0]):
    df['info_keywords'][i] = df['info_keywords'][i][:-1]
df.sample(5)

,year,type,genre,info,info_keywords
artists,,,,,
Charlie Puth,[2010],"[Solo, Male]","[POP, Electronica, Rap/Hiphop, R&B/Soul, EDM]","영화 ""분노의 질주 7""에 삽입된 폴 워커 추모곡 'See You Again'으로 ...","[앨범, You, 싱글, 그는, Meghan, 데뷔, 직접, 영화, 하는, Stev..."
EXO,[2010],"[Group, Male]","[Ballad, Dance, Rap/Hiphop, R&B/Soul, Elec...",현재 한국 대중음악계에서 최고의 인기를 누리고 있는 EXO는 태양계 밖의 외계 행성...,"[앨범, 타이틀곡, 1위를, 차트, 으로, 그룹의, 음악, MARZ, The, 수상..."
#안녕,[2010],"[Solo, Male]","[Ballad, Indie, Folk/Blues]","더필름의 객원 싱어로 출발해, '하나', '우리 다시 연락해요'를 히트시킨 #안녕은...","[안녕은, 처음, 소속사, 있다, 입사하였는데, 녹음을, 이용해, 더필, 가수, 있..."
Maroon 5,"[2000, 2010]","[Group, Male]","[POP, Rock/Metal, Electronica, Rap/Hiphop, ...",Maroon 5는 캘리포니아 주 출신의 7인조 팝 록 밴드이다. 그들의 데뷔 앨범인...,"[싱글, 앨범, 차트, 20, Be, 그들의, 빌보드, 역시, 만난, 1위, 이어,..."
SUGA,[2010],"[Solo, Male]",[Dance],SUGA는 빅히트 엔터테인먼트에 소속된 7인조 보이그룹 방탄소년단의 멤버로 그룹의 ...,"[LOVE, 방탄소년단의, 빅히트, 있다, 그룹의, 등을, 학교, 차트, 소속된, ..."


In [632]:
df['bagOfWords'] = ''
for index, row in df.iterrows():
    words = ''
    for col in ['year','type','genre', 'info_keywords']:
        words = words + ' '.join(row[col])+ ' '
    row['bagOfWords'] = words

textdf = df[['bagOfWords']]

In [633]:
textdf.sample(5)

,bagOfWords
artists,
최서현,2010 Solo Male Rock/Metal Rap/Hiphop Hash 더블...
김동률,1990 2000 2010 Solo Male Ballad 음반 많은 다시 20...
X1 (엑스원),2010 Group Male Ballad Dance 프로그램 비상 구성된 11인...
Post Malone,2010 Solo Male Rock/Metal Electronica Rap/H...
IZ*ONE (아이즈원),2010 Group Female Ballad Dance IZ 미니앨범 프로그램 ...


In [634]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
# Attempt1: CountVectorizer
vec = CountVectorizer()
X = vec.fit_transform(textdf['bagOfWords'])
pd.DataFrame(X.toarray(), columns=vec.get_feature_names()).sample(10)

,015b,10,100,100대,101,10cm는,10대들로만,10월,11,11월,...,히트곡,히트곡을,히트를,히트메이커,히트에,힘을,힘입어,힙합,힙합과,힙합의
134,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
121,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
177,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
122,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
45,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [635]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(X,X)
cosine_sim

array([[1.        , 0.20006253, 0.19223226, ..., 0.27508598, 0.18946619,
        0.26324906],
       [0.20006253, 1.        , 0.1339179 , ..., 0.14634146, 0.10559274,
        0.17972366],
       [0.19223226, 0.1339179 , 1.        , ..., 0.1339179 , 0.20291986,
        0.08458258],
       ...,
       [0.27508598, 0.14634146, 0.1339179 , ..., 1.        , 0.31677821,
        0.17972366],
       [0.18946619, 0.10559274, 0.20291986, ..., 0.31677821, 1.        ,
        0.111154  ],
       [0.26324906, 0.17972366, 0.08458258, ..., 0.17972366, 0.111154  ,
        1.        ]])

In [636]:
# Attempt2: TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vec2 = TfidfVectorizer()
X2 = vec2.fit_transform(textdf['bagOfWords'])
pd.DataFrame(X2.toarray(), columns=vec2.get_feature_names()).sample(10)

,015b,10,100,100대,101,10cm는,10대들로만,10월,11,11월,...,히트곡,히트곡을,히트를,히트메이커,히트에,힘을,힘입어,힙합,힙합과,힙합의
53,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
72,0.188283,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
59,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.217532,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
54,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
113,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.122706,0.0,0.0
140,0.000000,0.0,0.0,0.0,0.0,0.0,0.212478,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.112258,0.0,0.0
150,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.181032,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.114375,0.0,0.0
158,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
99,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
147,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.112975,0.0,0.0


In [637]:
cosine_sim2 = cosine_similarity(X2,X2)
cosine_sim2

array([[1.        , 0.06383762, 0.10242214, ..., 0.06693964, 0.04756623,
        0.0787217 ],
       [0.06383762, 1.        , 0.07603909, ..., 0.03327848, 0.04075401,
        0.04590525],
       [0.10242214, 0.07603909, 1.        , ..., 0.03167026, 0.07807314,
        0.01354128],
       ...,
       [0.06693964, 0.03327848, 0.03167026, ..., 1.        , 0.1010693 ,
        0.02353844],
       [0.04756623, 0.04075401, 0.07807314, ..., 0.1010693 , 1.        ,
        0.01328185],
       [0.0787217 , 0.04590525, 0.01354128, ..., 0.02353844, 0.01328185,
        1.        ]])

In [638]:
indices = pd.Series(textdf.index)
indices

0                               기리보이 (홍시영)
1                                       김필
2                                김소유 (김소희)
3                               김하온 (HAON)
4                                      김동률
5                               나플라(nafla)
6                                     뉴이스트
7                                 나얼 (유나얼)
8                                 넬 (NELL)
9                                       노을
10                                    드림캐쳐
11                                     다비치
12                                 다이나믹 듀오
13                               디오 (D.O.)
14                            동방신기 (TVXQ!)
15                       릴러말즈 (Leellamarz)
16                                    러블리즈
17                                로꼬 (권혁우)
18                                 린 (이세진)
19                             레이니 (Rainy)
20                            마마무(Mamamoo)
21                                    멜로망스
22                                   매드클라운
23         

In [639]:
def recommendations(artist, matrix):
    
    top10 = []
    sims = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == artist].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(matrix[idx]).sort_values(ascending = False)
#     print(score_series)
    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        top10.append(list(df.index)[i])
        
    return top10

In [640]:
recommendations('기리보이 (홍시영)', cosine_sim)

['Hash Swan',
 '한요한',
 'YUNHWAY',
 'Dok2 (이준경)',
 'XXXTENTACION (Jahseh Dwayne Onfroy)',
 '로꼬 (권혁우)',
 '폴킴',
 '태양 (동영배)',
 'BewhY (비와이)',
 '사이먼 도미닉 (정기석)']

In [641]:
recommendations('기리보이 (홍시영)', cosine_sim2)

['YUNHWAY',
 '한요한',
 'XXXTENTACION (Jahseh Dwayne Onfroy)',
 'Hash Swan',
 'BewhY (비와이)',
 '헤이즈 (Heize)',
 'Dok2 (이준경)',
 '김소유 (김소희)',
 'ZENE THE ZILLA',
 '로꼬 (권혁우)']